In [2]:
from fastai.vision.all import *
import cv2
import skimage as sm
import skimage.io
import tifffile
path = Path('/notebooks/deepLearningPaper/UNetCellDivision3Eval')

valid_fnames = (path/'valid.txt').read_text().split('\n')
path_im = path/'dat3/train_images'
path_lbl = path/'dat3/train_masks'
codes = np.loadtxt(path/'codes.txt', dtype=str)
# codes = np.loadtxt(path/'codes.txt', dtype=str)
fnames = get_image_files(path_im)
lbl_names = get_image_files(path_lbl)
get_msk = lambda o: path/f'dat3/train_masks'/f'{o.stem}_mask{o.suffix}'

camvid = DataBlock(blocks=(ImageBlock, MaskBlock(codes)),
                   get_items=get_image_files,
                   splitter=FileSplitter(path/'valid.txt'),
                   get_y=get_msk,
                   batch_tfms=[*aug_transforms(), Normalize.from_stats(*imagenet_stats)])

dls = camvid.dataloaders(path/f'dat3/train_images', bs=8)
dls.vocab = codes
name2id = {v:k for k,v in enumerate(codes)}
void_code = name2id['Void']
def acc_camvid(inp, targ):
  targ = targ.squeeze(1)
  mask = targ != void_code
  return (inp.argmax(dim=1)[mask]==targ[mask]).float().mean() 
opt = ranger
learn = unet_learner(dls, resnet34, metrics=acc_camvid, self_attention=True, act_cls=Mish, opt_func=opt).to_fp16()


device="cuda"
learn.load(f'UNetCellDivision3')
model = learn.model
model = model.to(device)

In [6]:
from cgitb import reset
import torch
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader
import os
from PIL import Image
from torch.utils.data import Dataset
import numpy as np
import skimage as sm
import skimage.io
from matplotlib import pyplot as plt
import tifffile
from fastai.vision.all import *
from skimage.feature import blob, blob_dog, blob_log, blob_doh
from os.path import exists

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 8
NUM_WORKERS = 2
IMAGE_HEIGHT = 512
IMAGE_WIDTH = 512
PIN_MEMORY = True

# util

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Creating directory. " + directory)

        
def sortDL(df, t, x, y):

    a = df[df["T"] == t - 2]
    b = df[df["T"] == t - 1]
    c = df[df["T"] == t + 1]
    d = df[df["T"] == t + 2]

    df = pd.concat([a, b, c, d])

    xMax = x + 13
    xMin = x - 13
    yMax = y + 13
    yMin = y - 13
    if xMax > 511:
        xMax = 511
    if yMax > 511:
        yMax = 511
    if xMin < 0:
        xMin = 0
    if yMin < 0:
        yMin = 0

    dfxmin = df[df["X"] >= xMin]
    dfx = dfxmin[dfxmin["X"] < xMax]

    dfymin = dfx[dfx["Y"] >= yMin]
    df = dfymin[dfymin["Y"] < yMax]

    return df


def intensity(vid, ti, xi, yi):

    [T, X, Y] = vid.shape

    vidBoundary = np.zeros([T, 552, 552])

    for x in range(X):
        for y in range(Y):
            vidBoundary[:, 20 + x, 20 + y] = vid[:, x, y]

    rr, cc = sm.draw.disk([yi + 20, xi + 20], 9)
    div = vidBoundary[ti][rr, cc]
    div = div[div > 0]

    mu = np.mean(div)

    return mu
        
    
def main():
    
    cwd = os.getcwd()
    filenames = os.listdir(cwd + "/dat_pred")
    if ".DS_Store" in filenames:
        filenames.remove(".DS_Store")
    if ".ipynb_checkpoints" in filenames:
        filenames.remove(".ipynb_checkpoints")
    filenames.sort()

    label=0
    for filename in filenames:
        print(filename)
        path_to_file = f"dat_output/{filename}/dfDivisions{filename}.pkl"
        focus = sm.io.imread(f"dat_pred/{filename}/focus{filename}.tif").astype(int)
        T = focus.shape[0]-4
        if False == exists(path_to_file):
            createFolder(f"dat_output/{filename}")
            for t in range(T):
                out = learn.predict(f'dat_pred/{filename}/{filename}_{t}.tif')[2][1]
                out = np.asarray(out*255, "uint8")
                tifffile.imwrite(f"dat_output/{filename}/pred_{filename}_{t}.tif", out)

            vid = np.zeros([T, 512, 512])

            img = np.zeros([552, 552])
            vid[0] = sm.io.imread(f"dat_output/{filename}/pred_{filename}_{0}.tif").astype(int)
            img[20:532, 20:532] = vid[0]
            blobs = blob_log(img, min_sigma=10, max_sigma=25, num_sigma=25, threshold=30)
            blobs_logs = np.concatenate((blobs, np.zeros([len(blobs), 1])), axis=1)

            for t in range(1, T):
                img = np.zeros([552, 552])        
                vid[t] = sm.io.imread(f"dat_output/{filename}/pred_{filename}_{t}.tif").astype(int)
                img[20:532, 20:532] = vid[t]
                blobs = blob_log(img, min_sigma=10, max_sigma=25, num_sigma=25, threshold=30)
                blobs_log = np.concatenate((blobs, np.zeros([len(blobs), 1]) + t), axis=1)
                blobs_logs = np.concatenate((blobs_logs, blobs_log))


            _df = []
            for blob in blobs_logs:
                y, x, r, t = blob
                mu = intensity(vid, int(t), int(x - 20), int(y - 20))

                _df.append(
                    {
                        "Label": label,
                        "T": int(t + 1),
                        "X": int(x - 20),
                        "Y": 532 - int(y),  # map coords without boundary
                        "Intensity": mu,
                    }
                )
                label += 1

            df = pd.DataFrame(_df)
            df.to_pickle(f"dat_output/{filename}/_dfDivisions{filename}.pkl")
            dfRemove = pd.read_pickle(f"dat_output/{filename}/_dfDivisions{filename}.pkl")

            for i in range(len(df)):
                ti, xi, yi = df["T"].iloc[i], df["X"].iloc[i], df["Y"].iloc[i]
                labeli = df["Label"].iloc[i]
                dfmulti = sortDL(df, ti, xi, yi)
                dfmulti = dfmulti.drop_duplicates(subset=["T", "X", "Y"])

                if len(dfmulti) > 0:
                    mui = df["Intensity"].iloc[i]
                    for j in range(len(dfmulti)):
                        tj, xj, yj = (
                            dfmulti["T"].iloc[j],
                            dfmulti["X"].iloc[j],
                            dfmulti["Y"].iloc[j],
                        )
                        labelj = dfmulti["Label"].iloc[j]
                        muj = dfmulti["Intensity"].iloc[j]

                        if mui < muj:
                            indexNames = dfRemove[dfRemove["Label"] == labeli].index
                            dfRemove.drop(indexNames, inplace=True)
                        else:
                            indexNames = dfRemove[dfRemove["Label"] == labelj].index
                            dfRemove.drop(indexNames, inplace=True)

            dfDivisions = dfRemove.drop_duplicates(subset=["T", "X", "Y"])
            dfDivisions.to_pickle(f"dat_output/{filename}/dfDivisions{filename}.pkl")
            os.remove(f"dat_output/{filename}/_dfDivisions{filename}.pkl")

In [7]:
main()

Unwound18h19


KeyboardInterrupt: 

In [18]:
model = learn.model
from torchsummary import summary
summary(model, (3, 512, 512))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 256]           9,408
       BatchNorm2d-2         [-1, 64, 256, 256]             128
              ReLU-3         [-1, 64, 256, 256]               0
         MaxPool2d-4         [-1, 64, 128, 128]               0
            Conv2d-5         [-1, 64, 128, 128]          36,864
       BatchNorm2d-6         [-1, 64, 128, 128]             128
              ReLU-7         [-1, 64, 128, 128]               0
            Conv2d-8         [-1, 64, 128, 128]          36,864
       BatchNorm2d-9         [-1, 64, 128, 128]             128
             ReLU-10         [-1, 64, 128, 128]               0
       BasicBlock-11         [-1, 64, 128, 128]               0
           Conv2d-12         [-1, 64, 128, 128]          36,864
      BatchNorm2d-13         [-1, 64, 128, 128]             128
             ReLU-14         [-1, 64, 1